In [221]:
import numpy as np
from gym import Env as GymEnv
import gym

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

In [223]:
learning_rate = 0.0002
gamma = 0.98
class PIDpolicy(nn.Module):
    def __init__(self):
        super(PIDpolicy, self).__init__()
        self.data = []
        
        self.fc1 = nn.Linear(4,128)
        self.fc2 = nn.Linear(128,3)
        self.optimizer = optim.Adam(self.parameters(), lr=learning_rate)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        
        return x
    
    def put_data(self, item):
        self.data.append(item)
    
    def train_net(self):
        R = 0
        self.optimizer.zero_grad()
        for r, prob in self.data[::-1]:
            R = r+gamma*R
            loss = -torch.log(prob)*R
            loss.backward()
        self.optimizer.step()
        self.data = []

In [224]:
class BasicAgent(object):
    def __init__(self, action_space):
        self.action_space = action_space
    def initialize(self, state):
        pass
    def pretraining_act(self, state): # training 전 데이터 모으기용
        return self.action_space.sample()
    def training_acct(self, state): # training용
        return self.action_space.sample()
    def solving_act(self, state): # test용
        return self.action_space.sample()
    def pretraining_react(self, state, reward): #tr
        pass
    def training_react(self, state, reward):
        pass
    def solving_react(self, state, reward):
        pass  

In [225]:
class PIDControlAgent(BasicAgent):
    def __init__(self, action_space, fs, kp=1.2, ki=1.0, kd=0.001, set_angle=0):
        # action_space : gym.spaces : 가능한 액션
        # fs : sampling frequency. (Hz) == 50.
        # kp : gain of proportional controller
        # ki : gain of integral controller
        # kd : derivative controller
        super(PIDControlAgent, self).__init__(action_space)
        self.kp = kp
        self.ki = ki
        self.kd = kd
        
        self.set_angle = set_angle #원하는 각도 : 0도가 이상적
        self.tau = 1.0/fs
        
        self.p_term = 0.0
        self.i_term = 0.0
        self.d_term = 0.0
        
        self.p_o=0
        self.i_o=0
        self.d_o=0

        # cache
        self.output = 0.0
        self.err_prev = 0.0
                
    def update(self, v_in, v_fb):
        # v_in : input command : 원하는 각도
        # v_fb : feedback from observer : 현재 각도
        # output : output command??
        # u(t) = K_p e(t) + K_i \int_{0}^{t} e(t)dt + K_d {de}/{dt}
        err = v_in - v_fb # 0 - 현재각
        
        #Ziegler–Nichols method
        self.p_term = err
        self.i_term += err*self.tau
        self.d_term = (err - self.err_prev)*self.tau
        self.output = self.kp*self.p_term + self.ki*self.i_term + self.kd*self.d_term
        
        self.err_prev = err
        
        return self.output
        
    def choose_action(self, val):
        if val >= 0:
            action = 0
        else:
            action = 1
        return action
    
    def solving_act(self, state):
        output = self.update(self.set_angle, state[2])
        temp = self.choose_action(output)
        self.action = temp
        return self.action

In [226]:
class BasicSolver(object):
    def __init__(self, env=None, agent=None, policy=None,
                 skip_pretraining=False,
                 skip_training=False,
                 skip_solving=False):
        self.env = env
        self.agent = agent
        self.policy = policy
        # method rebinding (make each stage skippable)
        self.solve = skip(flag=skip_solving)(self.solve)
    def pretrain(self):
        pass
    def train(self):
        pass
    def solve(self):
        pass
    def run(self):
        self.solve()
    def terminate(self):
        self.env.close()

In [227]:
class CartPoleSolver(BasicSolver):
    def __init__(self,
                 solving_episodes=10,
                 max_steps = 200,
                 render_when_sovling=True,
                 **kwargs):
        super(CartPoleSolver, self).__init__(**kwargs)

        self.solving_episodes = solving_episodes
        self.max_steps = max_steps

        # flags control for rendering
        self.rws = render_when_sovling

    def solve(self):
        state = self.env.reset()
        self.agent.initialize(state)

        for i in range(10000):
            total_reward = 0
            done = False

            while not done:

                action = self.agent.solving_act(state)
                state, reward, done, info = self.env.step(action)
                k = self.policy(torch.from_numpy(state).float())
                self.policy.put_data((reward, k))
                kp = k[0].detach().numpy()
                ki = k[1].detach().numpy()
                kd = k[2].detach().numpy()
                self.agent.kp = kp
                self.agent.ki = ki
                self.agent.kd = kd

                total_reward += reward

                if done:
                    print(f'Kp:{self.agent.kp}, Ki:{self.agent.ki}, Kd:{self.agent.kd}')
                    print('Episode: {}'.format(i),
                          'Total reward: {}'.format(total_reward))
                    self.env.reset()
        self.env.close()


In [228]:
def pid_control_solver():
    env = gym.make('CartPole-v0')
    # NOTE: kp, ki, kd are tuned manually, they are not the optimal parameter
    # for this PID controller
    kp = np.random.randint(10)
    ki = np.random.randint(10)
    kp = np.random.randint(10)
    print('initial kp,ki,kp', kp,ki,kp)
    policy = PIDpolicy()
    
    agent = PIDControlAgent(env.action_space, 
                        env.metadata['video.frames_per_second'],
                        kp=kp, ki=ki, kd=kp)
    # NOTE: pretraining and training stage is not required for this solver
    solver = CartPoleSolver(env=env, agent=agent, policy=policy,
                            skip_pretraining=True,
                            skip_training=True)
    solver.run()

In [229]:
env = gym.make('CartPole-v0')

In [230]:
def main():
    pid_control_solver()

In [231]:
if __name__ == '__main__':
    main()

initial kp,ki,kp 2 1 2
Kp:0.0, Ki:0.0, Kd:0.37117713689804077
Episode: 0 Total reward: 200.0
Kp:0.0, Ki:0.0, Kd:0.376310259103775
Episode: 1 Total reward: 200.0
Kp:0.202073872089386, Ki:0.0, Kd:0.45195871591567993
Episode: 2 Total reward: 200.0
Kp:0.0, Ki:0.0, Kd:0.3807896673679352
Episode: 3 Total reward: 200.0
Kp:0.18656808137893677, Ki:0.0, Kd:0.44545629620552063
Episode: 4 Total reward: 200.0
Kp:0.028351999819278717, Ki:0.0, Kd:0.4059133231639862
Episode: 5 Total reward: 200.0
Kp:0.0, Ki:0.0, Kd:0.3619857132434845
Episode: 6 Total reward: 200.0
Kp:0.2074653059244156, Ki:0.0, Kd:0.4545881152153015
Episode: 7 Total reward: 200.0
Kp:0.0, Ki:0.0, Kd:0.342162549495697
Episode: 8 Total reward: 200.0
Kp:0.46882712841033936, Ki:0.0, Kd:0.6074446439743042
Episode: 9 Total reward: 200.0
Kp:0.19360563158988953, Ki:0.0, Kd:0.5949984788894653
Episode: 10 Total reward: 190.0
Kp:0.0, Ki:0.0, Kd:0.4372258186340332
Episode: 11 Total reward: 200.0
Kp:0.0, Ki:0.0, Kd:0.3670665919780731
Episode: 12 To

Kp:0.0, Ki:0.0, Kd:0.35400938987731934
Episode: 108 Total reward: 200.0
Kp:0.0, Ki:0.0, Kd:0.35344284772872925
Episode: 109 Total reward: 200.0
Kp:0.0, Ki:0.0, Kd:0.37607866525650024
Episode: 110 Total reward: 200.0
Kp:0.0, Ki:0.0, Kd:0.36988750100135803
Episode: 111 Total reward: 200.0
Kp:0.41966310143470764, Ki:0.0, Kd:0.5505453944206238
Episode: 112 Total reward: 200.0
Kp:0.17516955733299255, Ki:0.0, Kd:0.439785361289978
Episode: 113 Total reward: 200.0
Kp:0.0, Ki:0.0, Kd:0.34782952070236206
Episode: 114 Total reward: 200.0
Kp:0.06092424690723419, Ki:0.0, Kd:0.408597469329834
Episode: 115 Total reward: 200.0
Kp:0.12419846653938293, Ki:0.0, Kd:0.5677847862243652
Episode: 116 Total reward: 189.0
Kp:0.4611571133136749, Ki:0.0, Kd:0.5990505218505859
Episode: 117 Total reward: 200.0
Kp:0.0, Ki:0.0, Kd:0.3697255253791809
Episode: 118 Total reward: 200.0
Kp:0.0, Ki:0.0, Kd:0.36323490738868713
Episode: 119 Total reward: 200.0
Kp:0.13809388875961304, Ki:0.0, Kd:0.42573538422584534
Episode: 1

Kp:0.41761231422424316, Ki:0.0, Kd:0.5874384641647339
Episode: 216 Total reward: 200.0
Kp:0.41826295852661133, Ki:0.0, Kd:0.5833439826965332
Episode: 217 Total reward: 200.0
Kp:0.08785542100667953, Ki:0.0, Kd:0.43652209639549255
Episode: 218 Total reward: 200.0
Kp:0.008593067526817322, Ki:0.0, Kd:0.3990902006626129
Episode: 219 Total reward: 200.0
Kp:0.009893644601106644, Ki:0.0, Kd:0.39066603779792786
Episode: 220 Total reward: 200.0
Kp:0.0, Ki:0.0, Kd:0.37166136503219604
Episode: 221 Total reward: 200.0
Kp:0.0, Ki:0.0, Kd:0.37533900141716003
Episode: 222 Total reward: 200.0
Kp:0.0, Ki:0.0, Kd:0.38552650809288025
Episode: 223 Total reward: 200.0
Kp:0.3930102586746216, Ki:0.0, Kd:0.5296972990036011
Episode: 224 Total reward: 200.0
Kp:0.06698665022850037, Ki:0.0, Kd:0.42102667689323425
Episode: 225 Total reward: 200.0
Kp:0.0, Ki:0.0, Kd:0.37574049830436707
Episode: 226 Total reward: 200.0
Kp:0.0, Ki:0.0, Kd:0.3675038516521454
Episode: 227 Total reward: 200.0
Kp:0.0, Ki:0.0, Kd:0.3681963

Kp:0.2988329231739044, Ki:0.0, Kd:0.5077208876609802
Episode: 319 Total reward: 200.0
Kp:0.009528815746307373, Ki:0.0, Kd:0.39349564909935
Episode: 320 Total reward: 200.0
Kp:0.01738058030605316, Ki:0.0, Kd:0.38760003447532654
Episode: 321 Total reward: 200.0
Kp:0.0, Ki:0.0, Kd:0.3735593855381012
Episode: 322 Total reward: 200.0
Kp:0.0, Ki:0.0, Kd:0.36413127183914185
Episode: 323 Total reward: 200.0
Kp:0.0, Ki:0.0, Kd:0.3385634422302246
Episode: 324 Total reward: 200.0
Kp:0.0, Ki:0.0, Kd:0.36445263028144836
Episode: 325 Total reward: 200.0
Kp:0.0, Ki:0.0, Kd:0.5378499031066895
Episode: 326 Total reward: 196.0
Kp:0.2240324318408966, Ki:0.0, Kd:0.46418431401252747
Episode: 327 Total reward: 200.0
Kp:0.007815137505531311, Ki:0.0, Kd:0.463581383228302
Episode: 328 Total reward: 200.0
Kp:0.0, Ki:0.0, Kd:0.3689124882221222
Episode: 329 Total reward: 200.0
Kp:0.0, Ki:0.0, Kd:0.3600093424320221
Episode: 330 Total reward: 200.0
Kp:0.0, Ki:0.0, Kd:0.44065362215042114
Episode: 331 Total reward: 2

Kp:0.4224581718444824, Ki:0.0, Kd:0.5450568199157715
Episode: 423 Total reward: 200.0
Kp:0.2955942153930664, Ki:0.0, Kd:0.5090518593788147
Episode: 424 Total reward: 200.0
Kp:0.0, Ki:0.0, Kd:0.375784307718277
Episode: 425 Total reward: 200.0
Kp:0.40868663787841797, Ki:0.0, Kd:0.5652416348457336
Episode: 426 Total reward: 200.0
Kp:0.0, Ki:0.0, Kd:0.4070740044116974
Episode: 427 Total reward: 200.0
Kp:0.49537187814712524, Ki:0.0, Kd:0.649666428565979
Episode: 428 Total reward: 200.0
Kp:0.03528023511171341, Ki:0.0, Kd:0.38744112849235535
Episode: 429 Total reward: 200.0
Kp:0.0, Ki:0.0, Kd:0.3806355893611908
Episode: 430 Total reward: 200.0
Kp:0.4033403992652893, Ki:0.0, Kd:0.573920726776123
Episode: 431 Total reward: 200.0
Kp:0.3713822662830353, Ki:0.0, Kd:0.5460329055786133
Episode: 432 Total reward: 200.0
Kp:0.0, Ki:0.0, Kd:0.3625184893608093
Episode: 433 Total reward: 200.0
Kp:0.39861786365509033, Ki:0.0, Kd:0.5417534112930298
Episode: 434 Total reward: 200.0
Kp:0.16867035627365112, Ki

...

Kp:0.49700474739074707, Ki:0.0, Kd:0.6379127502441406
Episode: 9501 Total reward: 200.0
Kp:0.2127065360546112, Ki:0.0, Kd:0.457543283700943
Episode: 9502 Total reward: 200.0
Kp:0.0, Ki:0.0, Kd:0.3581908047199249
Episode: 9503 Total reward: 200.0
Kp:0.0, Ki:0.0, Kd:0.3822014033794403
Episode: 9504 Total reward: 200.0
Kp:0.0, Ki:0.0, Kd:0.36994868516921997
Episode: 9505 Total reward: 200.0
Kp:0.0, Ki:0.0, Kd:0.3826940357685089
Episode: 9506 Total reward: 200.0
Kp:0.3641869127750397, Ki:0.0, Kd:0.5407117009162903
Episode: 9507 Total reward: 200.0
Kp:0.0, Ki:0.0, Kd:0.3740283250808716
Episode: 9508 Total reward: 200.0
Kp:0.0, Ki:0.0, Kd:0.361034631729126
Episode: 9509 Total reward: 200.0
Kp:0.2665252089500427, Ki:0.0, Kd:0.4905537962913513
Episode: 9510 Total reward: 200.0
Kp:0.0, Ki:0.0, Kd:0.34871265292167664
Episode: 9511 Total reward: 200.0
Kp:0.0, Ki:0.0, Kd:0.3722926378250122
Episode: 9512 Total reward: 200.0
Kp:0.0, Ki:0.0, Kd:0.3858819007873535
Episode: 9513 Total reward: 200.0
Kp:

Kp:0.396246075630188, Ki:0.0, Kd:0.5812647342681885
Episode: 9606 Total reward: 200.0
Kp:0.0, Ki:0.0, Kd:0.37273433804512024
Episode: 9607 Total reward: 200.0
Kp:0.0, Ki:0.0, Kd:0.382894903421402
Episode: 9608 Total reward: 200.0
Kp:0.02722470462322235, Ki:0.0, Kd:0.4104739725589752
Episode: 9609 Total reward: 200.0
Kp:0.17130009829998016, Ki:0.0, Kd:0.4591558873653412
Episode: 9610 Total reward: 200.0
Kp:0.0, Ki:0.0, Kd:0.5290305614471436
Episode: 9611 Total reward: 200.0
Kp:0.05057002604007721, Ki:0.0, Kd:0.4777916371822357
Episode: 9612 Total reward: 200.0
Kp:0.11519895493984222, Ki:0.0, Kd:0.42940589785575867
Episode: 9613 Total reward: 200.0
Kp:0.0, Ki:0.0, Kd:0.3838554620742798
Episode: 9614 Total reward: 200.0
Kp:0.0526149682700634, Ki:0.0, Kd:0.3998245894908905
Episode: 9615 Total reward: 200.0
Kp:0.3292028307914734, Ki:0.0, Kd:0.5189639925956726
Episode: 9616 Total reward: 200.0
Kp:0.0, Ki:0.0, Kd:0.37164223194122314
Episode: 9617 Total reward: 200.0
Kp:0.0, Ki:0.0, Kd:0.36426

Kp:0.08624061942100525, Ki:0.0, Kd:0.5335676670074463
Episode: 9710 Total reward: 200.0
Kp:0.0, Ki:0.0, Kd:0.37364932894706726
Episode: 9711 Total reward: 200.0
Kp:0.0, Ki:0.0, Kd:0.36490875482559204
Episode: 9712 Total reward: 200.0
Kp:0.0, Ki:0.0, Kd:0.3816768229007721
Episode: 9713 Total reward: 200.0
Kp:0.2509719133377075, Ki:0.0, Kd:0.4811056852340698
Episode: 9714 Total reward: 200.0
Kp:0.15061676502227783, Ki:0.0, Kd:0.4362950026988983
Episode: 9715 Total reward: 200.0
Kp:0.08501929044723511, Ki:0.0, Kd:0.43233034014701843
Episode: 9716 Total reward: 200.0
Kp:0.2711184024810791, Ki:0.0, Kd:0.49198031425476074
Episode: 9717 Total reward: 200.0
Kp:0.0, Ki:0.0, Kd:0.3637315332889557
Episode: 9718 Total reward: 200.0
Kp:0.36594831943511963, Ki:0.0, Kd:0.528772234916687
Episode: 9719 Total reward: 200.0
Kp:0.0, Ki:0.0, Kd:0.37077051401138306
Episode: 9720 Total reward: 200.0
Kp:0.0, Ki:0.0, Kd:0.3731164336204529
Episode: 9721 Total reward: 200.0
Kp:0.1903509646654129, Ki:0.0, Kd:0.44

Kp:0.12349021434783936, Ki:0.0, Kd:0.45055273175239563
Episode: 9811 Total reward: 200.0
Kp:0.11435353755950928, Ki:0.0, Kd:0.4681478440761566
Episode: 9812 Total reward: 200.0
Kp:0.154421865940094, Ki:0.0, Kd:0.437497615814209
Episode: 9813 Total reward: 200.0
Kp:0.1226184070110321, Ki:0.0, Kd:0.44924870133399963
Episode: 9814 Total reward: 200.0
Kp:0.18546637892723083, Ki:0.0, Kd:0.4384915232658386
Episode: 9815 Total reward: 200.0
Kp:0.10117091238498688, Ki:0.0, Kd:0.42795053124427795
Episode: 9816 Total reward: 200.0
Kp:0.007398102432489395, Ki:0.0, Kd:0.38662293553352356
Episode: 9817 Total reward: 200.0
Kp:0.28922995924949646, Ki:0.0, Kd:0.499288409948349
Episode: 9818 Total reward: 200.0
Kp:0.0, Ki:0.0, Kd:0.36682263016700745
Episode: 9819 Total reward: 200.0
Kp:0.0, Ki:0.0, Kd:0.3611350655555725
Episode: 9820 Total reward: 200.0
Kp:0.0, Ki:0.0, Kd:0.3683251738548279
Episode: 9821 Total reward: 200.0
Kp:0.09162576496601105, Ki:0.0, Kd:0.4427233338356018
Episode: 9822 Total rewar

Kp:0.0, Ki:0.0, Kd:0.37029385566711426
Episode: 9911 Total reward: 200.0
Kp:0.5483495593070984, Ki:0.0, Kd:0.6881009936332703
Episode: 9912 Total reward: 200.0
Kp:0.0, Ki:0.0, Kd:0.3812720477581024
Episode: 9913 Total reward: 200.0
Kp:0.45441287755966187, Ki:0.0, Kd:0.614639937877655
Episode: 9914 Total reward: 200.0
Kp:0.24989622831344604, Ki:0.0, Kd:0.4818877577781677
Episode: 9915 Total reward: 200.0
Kp:0.0, Ki:0.0, Kd:0.36340320110321045
Episode: 9916 Total reward: 200.0
Kp:0.0, Ki:0.0, Kd:0.3745277523994446
Episode: 9917 Total reward: 200.0
Kp:0.07539598643779755, Ki:0.0, Kd:0.41065314412117004
Episode: 9918 Total reward: 200.0
Kp:0.15860624611377716, Ki:0.0, Kd:0.436013787984848
Episode: 9919 Total reward: 200.0
Kp:0.0, Ki:0.0, Kd:0.36692509055137634
Episode: 9920 Total reward: 200.0
Kp:0.31680411100387573, Ki:0.0, Kd:0.5110099911689758
Episode: 9921 Total reward: 200.0
Kp:0.0, Ki:0.0, Kd:0.3892894685268402
Episode: 9922 Total reward: 200.0
Kp:0.2940657436847687, Ki:0.0, Kd:0.612